In [2]:
# Import python libraries
#
import importlib
import pickle
import os
import torch

# Go one directory back, because all imports are done
# relative to the root of the project.
#
project_root = '..'
if 'change_directory_to_root' not in globals():
    change_directory_to_root = True
    os.chdir(project_root)

# Imports own modules.
#
import scripts.Visualization as Visualization
import scripts.ModelTrainer as ModelTrainer
import scripts.Utils as utils
import scripts.Simulation_config as Simulation_config
from scripts.Simulation_config import *
import scripts.Model as model
import scripts.ModelAdapter as ModelAdapter


/home/molu/miniconda3/envs/xlstm/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/home/molu/miniconda3/envs/xlstm'


In [16]:
importlib.reload(model)
importlib.reload(ModelTrainer)
importlib.reload(Simulation_config)
importlib.reload(utils)

# Run the whole simulation
#
train_all_models = False
if train_all_models:
    configs = Simulation_config.configs
    ModelTrainer.ModelTrainer().run(configs)

# Print the sumarized simulation results
#
utils.Evaluate_Models.print_results('scripts/outputs/all_train_histories_20241031_1138.pkl')


("Configuration: Config_of_one_run(modelSize='MEDIUM', doPretraining=True, "
 'doTransferLearning=True, '
 "aggregation_Count='data/london_loadprofiles_37households_each.pkl', "
 'nrOfComunities=20, trainingHistory=466, modelInputHistory=24, '
 "usedModels=('SyntheticLoadProfile', 'KNN', 'PersistencePrediction'), "
 'epochs=100)')
Latex Summary for this Configuration:  & 34.05 (7.92) & 17.13 (2.69) & 14.39 (1.45)
("Configuration: Config_of_one_run(modelSize='SMALL', doPretraining=True, "
 'doTransferLearning=True, '
 "aggregation_Count='data/london_loadprofiles_37households_each.pkl', "
 'nrOfComunities=20, trainingHistory=466, modelInputHistory=24, '
 "usedModels=('SyntheticLoadProfile', 'KNN', 'PersistencePrediction'), "
 'epochs=100)')
Latex Summary for this Configuration:  & 34.05 (7.92) & 17.13 (2.69) & 14.39 (1.45)
("Configuration: Config_of_one_run(modelSize='LARGE', doPretraining=True, "
 'doTransferLearning=True, '
 "aggregation_Count='data/london_loadprofiles_37households_eac

In [6]:
# Plot the losses over the training.
# (To see, if the models are trained stable.)
#
importlib.reload(utils)

plot_only_single_config = True
plotted_config = Config_of_one_run(ModelSize.MEDIUM, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._37_HOUSEHOLDS, 
    NrOfComunities._20, TrainingHistory._15_MONTH, ModelInputHistory._1_DAY, UsedModels.ALL, Epochs.DEFAULT)

utils.Evaluate_Models.plot_training_losses_over_epochs('scripts/outputs/all_train_histories.pkl',
                                                       plot_only_single_config,
                                                       plotted_config
                                                       )


Plotted Config:
("Config_of_one_run(modelSize='MEDIUM', doPretraining=True, "
 'doTransferLearning=True, '
 "aggregation_Count='data/london_loadprofiles_37households_each.pkl', "
 'nrOfComunities=20, trainingHistory=466, modelInputHistory=24, '
 "usedModels=('SyntheticLoadProfile', 'KNN', 'PersistencePrediction'), "
 'epochs=100)')


In [ ]:
# Evaluate specific model of given power profiles and given configurations
#

importlib.reload(Visualization)
importlib.reload(model)
importlib.reload(utils)
importlib.reload(ModelAdapter)

# Define a specific configuration
#
test_profile = "scripts/outputs/file_0.pkl"
with open(test_profile, 'rb') as f:
    (X, Y, modelAdapter) = pickle.load(f)
path_to_trained_parameters = 'scripts/outputs/all_trained_models.pth'
model_type = 'SyntheticLoadProfile'
num_of_features = X['train'].shape[2]
myConfig = Config_of_one_run(ModelSize.MEDIUM, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._37_HOUSEHOLDS, 
                NrOfComunities._20, TrainingHistory._2_MONTH, ModelInputHistory._1_DAY, UsedModels.ALL, Epochs.DEFAULT)

# Plot the chosen model
#
myModel = utils.Deserialize.get_trained_model(path_to_trained_parameters, model_type, test_profile, myConfig, num_of_features)
plotlyApp = Visualization.PlotlyApp(X, Y, myModel, modelAdapter, None, 'UTC+00:00')
plotlyApp.run(myport=8060)


---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
File ~/loadforecasting/scripts/Visualization.py:132, in PlotlyApp.update_date_plot(
    self=<scripts.Visualization.PlotlyApp object>,
    selected_dataset='train',
    selected_date=1
)
    131 else:
--> 132     Y_pred = self.model_plot.predict(X_selected, Y_selected)
        X_selected = tensor([[[ 2.6708, -0.4160, -0.4160,  ...,  1.2889,  0.7238,  0.9566],
         [ 2.6708, -0.4160, -0.4160,  ...,  1.2889,  1.1622,  0.9572],
         [ 2.6708, -0.4160, -0.4160,  ...,  1.3688,  1.3815,  0.9586],
         ...,
         [-0.3744,  2.4037, -0.4160,  ..., -0.8665, -0.8593, -1.0390],
         [-0.3744,  2.4037, -0.4160,  ..., -0.8665, -0.8593, -1.0390],
         [-0.3744,  2.4037, -0.4160,  ..., -0.8665, -0.8593, -1.0390]],

        [[-0.3744,  2.4037, -0.4160,  ...,  1.3688, -0.4209,  0.9830],
         [-0.3744,  2.4037, -0.4160,  ...,  

In [5]:
# Store specific predictions, for the paper 'use case' section.
#

import numpy as np
importlib.reload(model)
importlib.reload(utils)

# Define a specific configuration
#
test_profile = "scripts/outputs/file_0.pkl"
with open(test_profile, 'rb') as f:
    (X, Y, modelAdapter) = pickle.load(f)
path_to_trained_parameters = 'scripts/outputs/all_trained_models.pth'
num_of_features = X['train'].shape[2]
myConfig = Config_of_one_run(ModelSize.MEDIUM, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._37_HOUSEHOLDS, 
                NrOfComunities._20, TrainingHistory._15_MONTH, ModelInputHistory._1_DAY, UsedModels.ALL, Epochs.DEFAULT)
P_el_predicted = modelAdapter.deNormalizeY(Y['test']).flatten()

# Store the predicted power profiles
#
for model_type in Simulation_config.UsedModels.ALL:
    my_Model = utils.Deserialize.get_trained_model(path_to_trained_parameters, model_type, test_profile, myConfig, num_of_features)
    Y_pred = torch.Tensor(modelAdapter.deNormalizeY(my_Model.predict(X['test'], Y['test'])).flatten())    
    P_el_predicted = torch.cat([P_el_predicted, Y_pred], dim=0)    

startdate = modelAdapter.getStartDateFromIndex('test', 0)
file_path = "scripts/outputs/example_predictions.pkl"
with open(file_path, 'wb') as file:
    pickle.dump((P_el_predicted, startdate), file)

print(P_el_predicted.shape)
print(startdate)


RuntimeError: X1 and X2 must have the same number of columns. X1: 3024 X2: 864

In [ ]:
# Print the model parameter sizes
#

importlib.reload(model)

model_types = ['xLSTM', 'LSTM', 'Transformer']
for model_type in model_types:
    for model_size in ['SMALL', 'MEDIUM', 'LARGE']:

        # Test, if the model is run-able
        num_of_features = 18
        m = model.Model(model_type, model_size, num_of_features)
        x = torch.zeros((7, 24*22, num_of_features))
        m.my_model(x)

        # Print the model's parameter count
        nr_of_parameters = m.get_nr_of_parameters(do_print=False)
        print(f"{model_type} - {model_size} has {nr_of_parameters} parameters.")


xLSTM - SMALL has 19791 parameters.
xLSTM - MEDIUM has 26411 parameters.
xLSTM - LARGE has 97679 parameters.
LSTM - SMALL has 17463 parameters.
LSTM - MEDIUM has 44031 parameters.
LSTM - LARGE has 104831 parameters.
Transformer - SMALL has 24423 parameters.
Transformer - MEDIUM has 50863 parameters.
Transformer - LARGE has 113351 parameters.


In [ ]:

class MeinModul():
    def __init__(self):
        self.custom_var1 = 1

# Saving the model
model = MeinModul()
torch.save(model, "model_with_custom_vars.pth")

# Loading the model
model1 = MeinModul()
state_dict = torch.load("model_with_custom_vars.pth")
# model.load_state_dict(state_dict)

model1.__dict__


{'custom_var1': 1}